# Jorual DAO: A New Framework for DeSci in Web 3.0

In [45]:
from math import floor
import pandas as pd
import hashlib
from random import randint

In [51]:

class IReview():
  address: int
  rating: int

  def __init__(self, address:str, rating:int) -> None:
    self.address = address
    self.rating = rating

class IDistribution():
  address: int
  token: int

  def __init__(self, address:str, token:int) -> None:
    self.address = address
    self.token = token



class ArticleDAO:
  _distribution:list[IDistribution]
  _articleID = None
  _authors:list[str]
  _reviewers:list[IReview]
  _log = pd.DataFrame()

  def __init__(self, articleID:int, authors:list[str]):
    self._articleID = articleID
    self._authors = authors
    self._reviewers = list()
    self._distribution = list()
  def review(self, reviews:list[IReview]):
    token = 0
    for review in reviews:
      token += review.rating
      self._distribution.append(IDistribution(review.address, (6 - review.rating) * 100))
    author_len = len(self._authors)

    token = token * 100
    weight = dict()
    weight_len = 0
    for i, author in enumerate(self._authors):
      n = author_len - i
      weight[i] = n
      weight_len += n
    for i, author in enumerate(self._authors):
      self._distribution.append(IDistribution(author, floor(token * (weight[i] / weight_len))))
  def download(self, address:str):
    user = next((obj for obj in self._distribution if obj.address == address), None)
    if user:
      user.token = user.token + 1
    else:
      self._distribution.append(IDistribution(address, 1))
    for authorAddress in self._authors:
      tmp = next((obj for obj in self._distribution if obj.address == authorAddress), None)
      tmp.token = tmp.token + 2
    self.record()
  def cite(self, address:str):
    user = next((obj for obj in self._distribution if obj.address == address), None)
    if user:
      user.token = user.token + 2
    else:
      self._distribution.append(IDistribution(address, 2))
    for authorAddress in self._authors:
      tmp = next((obj for obj in self._distribution if obj.address == authorAddress), None)
      tmp.token = tmp.token + 1
    self.record()
  def record(self):
    data = {}
    for obj in self._distribution:
      data[obj.address] = [obj.token]
    self._log = pd.concat([self._log, pd.DataFrame(data)], ignore_index=True)
  def emulate(self, download:int, cite:int):
    arr = []
    for i in range(download):
      arr.append({'address':'u' + str(i), 'type':'d'})
    for i in range(cite):
      index = randint(0, len(arr) - 1)

      arr.insert(index, {'address':'u' + str(randint(0,index)), 'type':'c'})

    for d in arr:
      if d['type'] == 'd':
        self.download(d['address'])
      else:
        self.cite(d['address'])
  def show(self):
    return self._log

In [53]:
article = ArticleDAO(1, ['a1','a2','a3'])
article.review([IReview(address='r1', rating=5), IReview(address='r2', rating=4)])
article.emulate(10, 1)

article.show()

,r1,r2,a1,a2,a3,u0,u1,u2,u3,u4,u5,u6,u7,u8,u9
0,100,200,452,302,152,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100,200,454,304,154,1,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,100,200,456,306,156,1,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,100,200,458,308,158,1,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
4,100,200,460,310,160,1,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN
5,100,200,462,312,162,1,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN
6,100,200,464,314,164,1,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN
7,100,200,465,315,165,1,3.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN
8,100,200,467,317,167,1,3.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN
9,100,200,469,319,169,1,3.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN
